In [1]:
import os
import time
import selenium
import pandas as pd

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [29]:
def _open_chromedriver(url, download_directory=r'C:\Users\lukar\Desktop\Sports Analytics\NBA Ball Handle Rate\data\starting-lineups'):
    '''
    returns browser object of chromedriver with a specified download directory
    '''
    # create chromeOptions object
    chrome_options = Options()
    
    prefs = {"download.default_directory" : download_directory}
    chrome_options.add_experimental_option("prefs",prefs)
    
    browser = webdriver.Chrome(options=chrome_options)
    time.sleep(0.5)
    
    browser.get(url)
    time.sleep(0.5)
    
    return browser

def _scrape_espn_starting_lineups(browser, filepath='data/starting-lineups/', season="2022-23"):
    # find table parent of the element
    table_parent = browser.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[2]/div[3]/div[1]/div[1]/div[1]/table')
    
    # find tbody element within parent
    tbody = table_parent.find_element(By.TAG_NAME, 'tbody')
    
    # iterate over all the table rows & extract cell data; but ignore stathead/colhead
    rows = tbody.find_elements(By.XPATH, './tr[not(contains(@class, "stathead") or contains(@class, "colhead"))]')

    columns = ['TEAM','PG','SG','SF','PF','C']
    data = []

    # iterate over all the table rows and extract cell data text
    for row in rows:
        # create empty lst to store player data
        lineups = []

        # get all the cells in the row
        cells = row.find_elements(By.TAG_NAME, 'td')

        # iterate over the cells and extract the cell text
        for cell in cells:
            lineups.append(cell.text)

        # append player data to list
        data.append(lineups)
        
    lineups_df = pd.DataFrame(data, columns=columns)
    
    lineups_df.to_csv(filepath + '{}_starting_lineups.csv'.format(season), index=False)

In [27]:
browser = _open_chromedriver(url="http://www.espn.com/nba/depth")

In [30]:
_scrape_espn_starting_lineups(browser)